In [3]:
import os, json, torch, Config, DataLoader

Data = DataLoader.JsonLoader().Get_Data()

In [5]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"trust_remote_code": True, 'device': device},
    encode_kwargs = {"normalize_embeddings": True},
    )

/home/iec/anaconda3/envs/agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/iec/anaconda3/envs/agent/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/iec/anaconda3/envs/agent/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant

qdrant_client = QdrantClient(
    url="https://816728f4-d70e-4ae8-9917-4888eb6bc5d1.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="Wmy6vlu1kICXewNIBXG6ce6JS8z_NKxp38xYf_IFQErhWZ9AIZUWHg",
)
database = Qdrant(client=qdrant_client, embeddings=embeddings, collection_name="Ami")

In [9]:
# qdrant = Qdrant.from_documents(
#     Data,
#     embeddings,
#     prefer_grpc=False,
#     collection_name="Ami",
#     api_key="Wmy6vlu1kICXewNIBXG6ce6JS8z_NKxp38xYf_IFQErhWZ9AIZUWHg",
#     url="https://816728f4-d70e-4ae8-9917-4888eb6bc5d1.us-east4-0.gcp.cloud.qdrant.io:6333",
# )
# # db = Qdrant(client=qdrant_client, embeddings=embedding_function, collection_name=cfg.QDRANT_COLLECTION_NAME)

In [7]:
retriever = database.as_retriever(search_kwargs={"k": 20})
text = retriever.get_relevant_documents("email thầy trần tiến công là gì?")


In [11]:
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import ContextualCompressionRetriever

model_name = "BAAI/bge-reranker-base"
model_kwargs = {'device': 'cuda'}
hf = HuggingFaceCrossEncoder(
    model_name=model_name,
    model_kwargs=model_kwargs
)
compressor = CrossEncoderReranker(model=hf, top_n=4)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [13]:
text = compression_retriever.get_relevant_documents("Email thầy Trần Tiến Công là gì?")
text

[Document(page_content='TS. Trần Tiến Công \nHọ và tên: \n \nTRẦN TIẾN CÔNG \n \nĐịa chỉ làm việc: \n \nHọc viện Công ngh ệ Bưu chính Vi ễn thông \n \nEmail: \n \ncongtt@ptit.edu.vn  \n \nLink Profile:', metadata={'Source': 'Trang 1, Tài Liệu: câu lạc bộ - CLB Kinh Tế', '_id': '791d58e0-e0c5-47a7-9154-377d444751c7', '_collection_name': 'Ami'}),
 Document(page_content='Youtube: https://www.youtube.com/channel/UCdOcZv16XwUi7bhawqPjV9g  \n    Email: clblaptrinhptit@gmail.com  \n       Phone: 0964 002 140', metadata={'Source': 'Trang 1, Tài Liệu: câu lạc bộ - CLB Kinh Tế', '_id': '20443fb7-2159-4cbc-bac2-fd88eb5d82b9', '_collection_name': 'Ami'}),
 Document(page_content='thoại: \xa0\xa0\xa0 028.38297220 Fanpage: \xa0\xa0\xa0\xa0\xa0\xa0 https://www.facebook.com/ptithochiminh Email: \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 tuyensinh@ptithcm.edu.vn', metadata={'Date': '24/07/2023', 'Source': 'https://portal.ptit.edu.vn/hoc-vien-cong-nghe-buu-chinh-vien-thong-thong-bao-nguong-dam-bao-chat

In [14]:
from langchain_community.retrievers import BM25Retriever

_Content = [doc.page_content for doc in Data]
bm25_retriever = BM25Retriever.from_texts(_Content, metadatas=[doc.metadata for doc in Data])
bm25_retriever.k = 20

In [15]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5])


In [16]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

In [18]:
text = compression_retriever.get_relevant_documents("Giám Đốc Học Viện Là Ai?")
text

[Document(page_content='BAN GIÁM Đ ỐC HỌC VIỆN CÔNG NGH Ệ BƯU CHÍNH VI ỄN THÔNG \nGiám đốc Học viện PGS.TS. Đ ặng Hoài B ắc', metadata={'Source': 'Trang 1, Tài Liệu: câu lạc bộ - CLB Kinh Tế', '_id': '22d01535-780b-4ea3-9c5b-170174c17bbf', '_collection_name': 'Ami'}),
 Document(page_content='II. BAN GIÁM Đ ỐC HỌC VIỆN: \n1. Giám đ ốc Học viện: PGS.TS. Đ ặng Hoài B ắc \nĐịa chỉ văn phòng: 122, Hoàng Qu ốc Việt, Cầu Giấy, Hà Nội \nĐịa chỉ văn phòng: Km10 –Đường Nguyễn Trãi, Hà Đông, Hà N ội. \n2. Phó Giám đ ốc: TS. Vũ Tu ấn Lâm \nĐịa chỉ văn phòng: Km10 –Đường Nguyễn Trãi, Hà Đông, Hà N ội. \n3. Phó Giám đ ốc: PGS.TS. Tr ần Quang Anh  \nĐịa chỉ văn phòng: 122, Hoàng Qu ốc Việt, Cầu Giấy, Hà Nội. \n4. Phó Giám đ ốc: TS. Tân H ạnh (phụ trách Học viện Cơ sở tại TP. Hồ Chí \nMinh) \nĐịa chỉ văn phòng: S ố 11, Nguy ễn Đình Chi ểu, Quận 1, TP. H ồ Chí Minh', metadata={'Source': 'Trang 1, Tài Liệu: câu lạc bộ - CLB Kinh Tế', '_id': '94f4d25e-c3cd-484c-ae7b-04f27b4d583d', '_collection_name': 'Am

: 